In [24]:
import numpy as np
from ncon import ncon
import scipy
import matplotlib.pyplot as plt

In [11]:
# Generate sample data 
nQbLabel = 2
labelDim = 2**nQbLabel

nCopies = 2
nQb = nQbLabel * nCopies
stateDim = 2**nQb

nClasses = 3 # Number of classes
nPerClass = 10 # Number of states per class
N = nClasses * nPerClass

In [22]:
# Generate state data and label data
states = np.zeros((N, labelDim))
labels = np.zeros(N)
for n in range(nClasses):
    v0 = np.random.randn(labelDim) 
    states[n*nPerClass: (n+1)*nPerClass] = v0 + 0.01*np.random.randn(nPerClass, labelDim)
    labels[n*nPerClass: (n+1)*nPerClass] = n
    
# Make sure states are normalised
states = states / np.linalg.norm(states, axis=1)[:, np.newaxis]

In [ ]:
def costTanh(states, labels, V):
    '''
    Calculate the Tanh Cost function for a given V. 
    '''
    
    V

In [ ]:
def gradientDescentTanh(states, labels, lr=1e-1, V0=None, maxIter=1e3, verbose=True):
    '''
    Optimise Tanh cost function using gradient descent. 
    
        Parameters
        ----------
        states : np.array (N, stateDim)
                 States representing input data. 
        labels : np.array (N, )
                 Labels assigned to states given as a number. 
        lr : float
             Learning ratr / step size during gradient descent. 
        V0 : np.array (stateDim, stateDim)
             Initial guess for V that optimises the tanh cost function. 
        maxIter : int
                  Maximum number of steps          
    '''
    stateDim = states.shape[1]
    
    if V0 is None:
        V0 = np.random.randn(stateDim, stateDim) + 1j*np.random.randn(stateDim, stateDim)
        V0, _ = scipy.linalg.polar(V0)
        
    g = gradientTanh(states, labels, V0)
    V = V0
    
    i = 0
    
    while i < maxIter:
        # Gradient descent step
        V = V - lr*g
        
        # Enforce unitarity
        V, _ = scipy.linalg.polar(V)
        
        i += 1
        
        if verbose and not(i%50):
            cost = costTanh(states, labels, V)
            acc = accuracyV(states, labels, V)
            print('iteration:\t{:d}\tcost:\t{:12f}\taccuracy:\t{:.4f}\tgradient norm:\t{:.4e}'.format(i, cost, acc, np.linalg.norm(g)))
            
        g = gradientTanh(states, labels, V)
        
        if i > maxIter: 
            print('Warning: Reached max iterations.')
            break
            
    
    cost = costTanh(states, labels, V)
    acc = accuracyV(states, labels, V)
    return V, cost, acc